In [ ]:
!conda create -n ftudd  --file requirements_chem.txt -c pytorch -c rdkit -c conda-forge -c rmg -y
# This will take a couple of minutes.

In [ ]:
%%bash
source ~/.bashrc
conda activate ftudd
ipython kernel install --name "ftudd" --user
# after runnning this cell restart the notebook with the ftudd kernel

### Grover model

Code and pretrained weights are available from here: https://github.com/tencent-ailab/grover. We will use a fork of it where I already fixed a bug in the code base.

Implementation of Yu et al., Self-Supervised Graph Transformer on Large-Scale Molecular Data, NeurIPS 2020

Grover is an instance of a graph neural network. It is trained in a self-supervised way, i.e. from unlabeled training data, and creates an embedding of a molecule. It can be fine-tuned for downstream tasks.

In [ ]:
# clone Grover repository
!git clone https://github.com/emdgroup/grover.git ../grover
!mkdir ../grover/data/
!wget https://ai.tencent.com/ailab/ml/ml-data/grover-models/pretrain/grover_large.tar.gz -O ../grover/data/grover_large.tar.gz
!tar -xzf ../grover/data/grover_large.tar.gz -C ../grover/data/
import sys
sys.path.append('../grover')

In [ ]:
# Grover has a command line interface, let us use it to generate an embedding of a sample molecule
import pandas as pd
smiles = ['CC(=O)O', 'CCCCCC']
pd.DataFrame({'SMILES': smiles}).to_csv('test_smiles.csv', index=False)

In [ ]:
%%bash
source ~/.bashrc
conda activate ftudd
python ../grover/scripts/save_features.py --data_path test_smiles.csv --save_path test_features.npz --features_generator rdkit_2d_normalized --restart 
python ../grover/main.py fingerprint --data_path test_smiles.csv --features_path test_features.npz --checkpoint_path ../grover/data/grover_large.pt --fingerprint_source both --output test_fingerprints.npz

In [ ]:
# load the fingerprints
import numpy as np
fingerprints = np.load('test_fingerprints.npz')["fps"]
print(fingerprints)
print(fingerprints.shape)

### ChemBERTa model

ChemBERTa is based on the BERT NLP model and treats SMILES strings as text that can be modeled. Most NLP models are nicely wrapped by the Huggingface transformer library and hence, we can leverage their API. Further details on ChemBERTa can be found in the paper:

Chithrananda et al., ChemBERTa: Large-Scale Self-Supervised Pretraining for Molecular Property Prediction, arXiv 2020

or at Github: https://github.com/seyonechithrananda/bert-loves-chemistry

In [ ]:
# Download Chemberta model
from transformers import AutoTokenizer, AutoModelForMaskedLM
chemberta_model_name = 'seyonec/ChemBERTa-zinc-base-v1'
chemberta_tokenizer = AutoTokenizer.from_pretrained(chemberta_model_name)
chemberta_model = AutoModelForMaskedLM.from_pretrained(chemberta_model_name)

In [ ]:
import torch
def embed_smiles(smiles, tokenizer, model, layers):
    """
    Returns the embedding of a SMILES string.
    """
    # Get the tokenized input
    tokenized_input = tokenizer(smiles, return_tensors='pt')
    # Get the embedding
    with torch.no_grad():
        output = model(**tokenized_input, output_hidden_states=True)
    # Return the embedding
    states = torch.stack([output.hidden_states[l] for l in layers]).mean([1,2]).view(-1)
    return states.detach().numpy()

test_embedding = embed_smiles('CC(=O)O', chemberta_tokenizer, chemberta_model, [-1])
print(test_embedding.shape)
print(test_embedding[:10])

In [ ]:
# Load AqSolDB data 
# Alternatively, you can use a dataset from here: https://github.com/GLambard/Molecules_Dataset_Collection/tree/master/latest
import pandas as pd
df_aqsol = pd.read_csv('data_chem/curated-solubility-dataset.csv')
print(df_aqsol.shape)
print(df_aqsol.head(4))
smiles = df_aqsol['SMILES'].values
targets = df_aqsol['Solubility'].values


### Your tasks
1. Create embeddings for the molecules in the AqSolDB dataset using both the pretrained ChemBERTa model as well as the Grover model
2. Train a suitable scikit-learn model on top of these embeddings to predict the solubility
3. Experiment with this setting and summarize your findings

### The advanced stuff
4. Fine tune Grover and ChemBERTa on the AqSol prediction task 
5. Experiment and summarize your findings
For fine-tuning Grover have a look at their Github page. For fine-tuning a language model from Huggingface, see here: https://huggingface.co/docs/transformers/training